# Local Small CNN Model Trainer

Disclaimer: Initial code generated by MinRes AI (Open Web UI) model:gpt-5

Input data sorted by folders

In [3]:
#Verify GPU availability
!nvidia-smi

Wed Oct  8 10:56:35 2025       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.247.01             Driver Version: 535.247.01   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 3060        Off | 00000000:01:00.0 Off |                  N/A |
| 38%   31C    P8              14W / 170W |     77MiB / 12288MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [1]:
# import extensions
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

## 1. Data pipeline

Some preprocessing followed by piping data into data sets for training and validation

### 1.1 Transformations
preprocesing input data (e.g. resizing, grascale, etc)


In [15]:

transform = transforms.Compose([
    transforms.Grayscale(),              # if your digits are greyscale
    transforms.Resize((28, 28)),         # standardise size
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,)) # normalise to [-1, 1]
])

### 1.2 Download Data
I download my data from my G Drive

In [17]:
# everything here is untracked by .gitignore btw
import gdown

url = "https://drive.google.com/uc?id=1slfq-_VI3d6QLGwa1zP1oRgxc5YZSWXQ"
output = "training_data/task3data.zip"
gdown.download(url, output, quiet=False)

!unzip -qo training_data/task3data.zip -d training_data/task3data/

train_data = datasets.ImageFolder(root='training_data/task3data/train', transform=transform)
val_data   = datasets.ImageFolder(root='training_data/task3data/valid', transform=transform)

train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
val_loader   = DataLoader(val_data, batch_size=32, shuffle=False)

Downloading...
From: https://drive.google.com/uc?id=1slfq-_VI3d6QLGwa1zP1oRgxc5YZSWXQ
To: /home/21502396/Searle_Jack_21502396/model_training/training_data/task3data.zip
100%|██████████| 4.52M/4.52M [00:00<00:00, 6.06MB/s]


## 2. simple CNN architecture
create them layers

In [18]:
class DigitCNN(nn.Module):
    def __init__(self, num_classes):
        super(DigitCNN, self).__init__()
        self.conv_layers = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),  # 14x14

            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2)   # 7x7
        )
        self.fc_layers = nn.Sequential(
            nn.Flatten(),
            nn.Linear(64 * 7 * 7, 128),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(128, num_classes)
        )

    def forward(self, x):
        x = self.conv_layers(x)
        x = self.fc_layers(x)
        return x


### 3. Set up model/optim/loss

In [19]:
num_classes = len(train_data.classes)  # should match number of digit folders
model = DigitCNN(num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


## 4. training loop

In [24]:
for epoch in range(60):  # adjust epochs as needed
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f"Epoch {epoch+1}, Loss: {running_loss/len(train_loader):.4f}")


Epoch 1, Loss: 0.0549
Epoch 2, Loss: 0.0682
Epoch 3, Loss: 0.0521
Epoch 4, Loss: 0.0643
Epoch 5, Loss: 0.0590
Epoch 6, Loss: 0.0406
Epoch 7, Loss: 0.0331
Epoch 8, Loss: 0.0387
Epoch 9, Loss: 0.0388
Epoch 10, Loss: 0.0190
Epoch 11, Loss: 0.0454
Epoch 12, Loss: 0.0565
Epoch 13, Loss: 0.0463
Epoch 14, Loss: 0.0276
Epoch 15, Loss: 0.0418
Epoch 16, Loss: 0.0362
Epoch 17, Loss: 0.0246
Epoch 18, Loss: 0.0499
Epoch 19, Loss: 0.0390
Epoch 20, Loss: 0.0274
Epoch 21, Loss: 0.0168
Epoch 22, Loss: 0.0202
Epoch 23, Loss: 0.0180
Epoch 24, Loss: 0.0197
Epoch 25, Loss: 0.0157
Epoch 26, Loss: 0.0122
Epoch 27, Loss: 0.0127
Epoch 28, Loss: 0.0272
Epoch 29, Loss: 0.0131
Epoch 30, Loss: 0.0056
Epoch 31, Loss: 0.0214
Epoch 32, Loss: 0.0188
Epoch 33, Loss: 0.0200
Epoch 34, Loss: 0.0193
Epoch 35, Loss: 0.0190
Epoch 36, Loss: 0.0111
Epoch 37, Loss: 0.0171
Epoch 38, Loss: 0.0101
Epoch 39, Loss: 0.0094
Epoch 40, Loss: 0.0141
Epoch 41, Loss: 0.0302
Epoch 42, Loss: 0.0237
Epoch 43, Loss: 0.0199
Epoch 44, Loss: 0.02

## 5. Validate Loop

In [25]:
# Validation loop
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in val_loader:
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
print(f"Validation Accuracy: {100 * correct / total:.2f}%")

Validation Accuracy: 88.67%


## 6. Save model

In [26]:
torch.save(model.state_dict(), "digit_classifier.pth")